<center>
    <font size ='+2'> Project: Predicting t$_{\text{LD,75}}$ Differences in Replicates</font>
    <br><br><font size ='+1'>by Andrew Tischhauser
    <br>Updated Jan. 20, 2021</font>
</center>

### Goals
There exist significant differences in t$_{\text{LD75}}$ and other metrics between identically prepared samples.

The question is whether or not we can predict t$_{\text{LD75}}$ differences between replicates based on early time measurements.

If so, can we also figure out the causes of these differences? (totally bonus)

### Table of Contents

- [Download Data](#download_data)
- [Fill Dataframes](#fill_dfs)
- [Characterizing Variance](#char_var)
- [Feature Explanation](#feature)
- [Modeling with LASSO Regression](#LASSO)

# Download Data (only run the first time using this notebook) <a class="anchor" id="download_data"></a>
Locally download the replicate study using google drive file stream.

Data Source:
- Preetham's pickle file loads replicates from Effort_Perovskites_3

Saving directly to a folder in Documents for convenience.

In [ ]:
import os
import shutil
from pathlib import Path

In [ ]:
import pickle
with open("Replication_runs", 'rb') as file:
    paths_list = pickle.load(file)

In [ ]:
# source_path = r'G:\Shared Drives\Effort_Perovskites\Machine_Learning\Data\Timeseries'
# source_path = r'G:\My Drive\Effort_Perovskites_3\Machine_Learning\Data\Timeseries'
# source_path = r'G:\.shortcut-targets-by-id\1QYKycHbBYmm38jHyBiIHndq6dxjR_RwH\Effort_Perovskites_3\Machine_Learning\Data\Timeseries'
dest_path = r'C:\Users\andre\Documents\replicate_data'

source_path_list = {r'G:\Shared Drives\Effort_Perovskites\Machine_Learning\Data\Timeseries',
                   r'G:\Shared Drives\Effort_Perovskites_2\Machine_Learning\Data\Timeseries',
                   r'G:\.shortcut-targets-by-id\1QYKycHbBYmm38jHyBiIHndq6dxjR_RwH\Effort_Perovskites_3\Machine_Learning\Data\Timeseries'}

for source_path in source_path_list:
    for folder_title in paths_list:
        # only create and populate the folder if the desired files exist in that folder
        make_path = dest_path + '\\' + '\\'.join(folder_title.split('/'))
        data_path = source_path + '\\' + '\\'.join(folder_title.split('/')) + '\\' + 'analyzed_data.csv'
        exp_path = source_path + '\\' + '\\'.join(folder_title.split('/')) + '\\' + 'experiment_info.json'
        sam_path = source_path + '\\' + '\\'.join(folder_title.split('/')) + '\\' + 'sample_info.json'
        if os.path.isfile(data_path) and os.path.isfile(exp_path) and os.path.isfile(sam_path):
            os.makedirs(make_path)
            shutil.copy(data_path, make_path)
            shutil.copy(exp_path, make_path)
            shutil.copy(sam_path, make_path)
            print(f'Downloaded {folder_title} .....')

In [ ]:
paths_list

# Fill Dataframes <a class="anchor" id="fill_dfs"></a>



**Load all possibly used features into df now, just assign desired features to X later.**

We have 33 data points (after removing errors) in this group of MAPI, 8 suns, 25°C, 60% RH, air replicates.
<br><br><br>
NOTE: Errors in quite a few of the runs, where no LD data was collected (not useable)
- Error in: 190916_PLVA_Perovskite_Degradation_Study/PL_T_MAPI_air_60RH_25C_8Suns_No_Contacts
- Error in: 200525_Replication_Study/PL_PC_T_MAPI_8Sun_25C_60RH_air_1_3_ctd
- Error in: 200525_Replication_Study/PL_PC_T_MAPI_8Sun_25C_60RH_air_1_3_ctd2
- Error in: 200525_Replication_Study/PL_PC_T_MAPI_8Sun_25C_60RH_air_2_2_ctd
- Error in: 200525_Replication_Study/PL_PC_T_MAPI_8Sun_25C_60RH_air_2_2_ctd2

Also one run where it never reached LD75:
- Error in: 200525_Replication_Study/PL_PC_T_MAPI_8Sun_25C_60RH_air_28_50x


In [1]:
import pandas as pd
import re
import json
import os
import numpy as np
import matplotlib.pyplot as plt
import math
import seaborn as sns

In [2]:
source_path = r'C:\Users\andre\Documents\replicate_data'

The following cell uses a regex object create a method that can parse through the composition string in case we need composition as a feature.

In [3]:
# Regex pattern: [1 or 2 letters][0 or 1 digits][period][1 or more digits], possibly repeated so use findall
comp_regex = re.compile(r'([a-zA-Z][a-zA-Z]?)(\d?\.\d+)')
def parse_sample_comp(comp_str, site, comp_regex):
    # Treat special cases
    if (comp_str == 'XXX' or comp_str == 'I.93Br.007') and site == 'X':
        comp_str = 'I.93Br.07'
    if comp_str == 'XXX' or comp_str == 'MA.5FA.43CS.007':  # These special cases were all part of tripcat with same comp, assumed 'XXX' to be the same and CS.007 assumed to be typo for CS.07 (since all other comps add to 1)
        comp_str = 'MA.5FA.43CS.07'

    # Remove whitespace and capitalize letters
    comp_str = ''.join(comp_str.split()).upper()

    # Break into list of tuples, ex. mo1 = [('MA', '0.5'), ('FA', '0.415'), ('CS', '0.085')]
    mo1 = comp_regex.findall(comp_str)

    species_list = []
    values_list = []

    for tup in mo1:
        species_list.append(tup[0])
        values_list.append(float(tup[1]))
    
    return [species_list, values_list]

The following cell defines our method used for taking a folder for one run of data and turning it into a row for our dataframe. Many potential features have already been created.

Notes:
- Should check that method of reading NSuns is correct

In [179]:
def get_single_run_data(folder_path, time_allowed):
    # This method takes a path to a single run folder, returns features and target as list
    data = pd.read_csv(folder_path + '\\analyzed_data.csv')
    LD_norm = data['Low Freq LD [norm]']
    Tr_norm = data['Transmitted Power [norm]']
    PL_norm = data['PLQY_xy0t0']/data['PLQY_xy0t0'][0]
    t = data['t']
    indexes = []
    values = []
    
    
    # GET t_LD75
    LD75 = 0.75 * LD_norm[0]
    for i, LDi in enumerate(LD_norm):
        if LDi <= LD75 and (LD_norm[i+1] <= LD75 or i == len(LD_norm) - 1): # or statement might help with ID'ing outliers
            t_LD75 = (LD75 - LD_norm[i-1])/(LD_norm[i] - LD_norm[i-1]) * (t[i] - t[i-1]) + t[i-1]
            t_LD75 /= 60 # convert to minutes from seconds
            break
        elif i == len(LD_norm) - 1:
            raise Exception(f'{folder_path} does not reach LD75.')
    
    if t_LD75 * 60 <= time_allowed:
        raise Exception(f'{folder_path} reaches LD75 before the end of the prediction window.')
    indexes.append('ln(t_LD75)')
    values.append(math.log(t_LD75))

    
    
    # GET TIME ZERO VIDEO FEATURES FROM CSV
    t_zero_feature_list = [
                    'xy0t0',
                    'PLQY_xy0t0',
                    'xy1t0',
                    'xy2t0',
                    'xy3t0',
                    'QFLS_xy0t0',
                    'frac_bright',
                    'cv_slopes']
    for feature_name in t_zero_feature_list:
        indexes.append(feature_name + ' t0')
        values.append(data[feature_name][0])

    # GET FIRST AND SECOND DERIVS OF Tr, PL, Ld AT t=0
    # Ask Tim what he thinks of this approach to basing derivatives on prediction window
    min_n_use = 2
    max_n_use = 5 # remove these caps
    min_n_use_der = 3
    max_n_use_der = 5 # remove
    tstep = t.iloc[1]-t.iloc[0]
    avail_pts = time_allowed // tstep + 1
    n_use = int(min(avail_pts, max_n_use))
    n_use_der = int(min(avail_pts, max_n_use_der))
    # Calculate slopes at t=0
    if n_use >= min_n_use:
        dTr0 = np.polyfit(np.arange(n_use)*tstep,Tr_norm.iloc[:n_use].values,1)[0]
        dPL0 = np.polyfit(np.arange(n_use)*tstep,PL_norm.iloc[:n_use].values,1)[0]
        dLd0 = np.polyfit(np.arange(n_use)*tstep,LD_norm.iloc[:n_use].values,1)[0]
        indexes.append('dTr0')
        values.append(dTr0)
        indexes.append('dPL0')
        values.append(dPL0)
        indexes.append('dLd0')
        values.append(dLd0)
    # Calculate curvature at t=0 (use finite differences)
    # SEPARATELY CALCULATE FOR THE RUN WITH TSTEP 600
    if n_use_der >= min_n_use_der:
        ddTr0 = np.mean(np.gradient(np.gradient(Tr_norm.iloc[:n_use_der].values, np.arange(n_use_der)*tstep), np.arange(n_use_der)*tstep))
        ddPL0 = np.mean(np.gradient(np.gradient(PL_norm.iloc[:n_use_der].values, np.arange(n_use_der)*tstep), np.arange(n_use_der)*tstep))
        ddLd0 = np.mean(np.gradient(np.gradient(LD_norm.iloc[:n_use_der].values, np.arange(n_use_der)*tstep), np.arange(n_use_der)*tstep))
        indexes.append('ddTr0')
        values.append(ddTr0)
        indexes.append('ddPL0')
        values.append(ddPL0)
        indexes.append('ddLd0')
        values.append(ddLd0)

 
    # GET TIMESERIES FEATURES (NORMALIZED TR, PL, LD AT POINTS DEPENDING ON TIME_ALLOWED, interpolated)
    try:
        # Even number minute points
        num_evens = time_allowed//120
        for even_min in range(1, num_evens + 1):
            ti = even_min * 120
            i = ti//tstep + 1
            indexes.append(f'Tr [norm], t={ti} s')
            values.append((ti - t[i-1])/(t[i] - t[i-1])*(Tr_norm[i] - Tr_norm[i-1]) + Tr_norm[i-1])
            indexes.append(f'PL [norm], t={ti} s')
            values.append((ti - t[i-1])/(t[i] - t[i-1])*(PL_norm[i] - PL_norm[i-1]) + PL_norm[i-1])
            indexes.append(f'Ld [norm], t={ti} s')
            values.append((ti - t[i-1])/(t[i] - t[i-1])*(LD_norm[i] - LD_norm[i-1]) + LD_norm[i-1])
        # Final time point (if odd minute)
        if time_allowed % 120 != 0:
            i = time_allowed//tstep + 1
            indexes.append(f'Tr [norm], t={time_allowed} s')
            values.append((time_allowed - t[i-1])/(t[i] - t[i-1])*(Tr_norm[i] - Tr_norm[i-1]) + Tr_norm[i-1])
            indexes.append(f'PL [norm], t={time_allowed} s')
            values.append((time_allowed - t[i-1])/(t[i] - t[i-1])*(PL_norm[i] - PL_norm[i-1]) + PL_norm[i-1])
            indexes.append(f'Ld [norm], t={time_allowed} s')
            values.append((time_allowed - t[i-1])/(t[i] - t[i-1])*(LD_norm[i] - LD_norm[i-1]) + LD_norm[i-1])
    except:
        print(f'{folder_path} does not run for the chosen amount of time, {time_allowed} s')
    
    
    # Get RH, atmosphere composition, and temperature
    with open(folder_path + '\\experiment_info.json') as exp_json:
        exp_info = json.load(exp_json)
        indexes.append('RH (%)')
        values.append(exp_info['Atmosphere_RH (%)'])
        indexes.append('Temp (C)')
        values.append(exp_info['Temperature (deg C)'])
        if exp_info['Atmosphere_O2 (%)'] != 21.0:
            raise Exception(f'{folder_path} is not in 21% O2.')
        indexes.append('Atm O2 (%)')
        values.append(exp_info['Atmosphere_O2 (%)'])
        try:
            NSuns = exp_info['Stress Intensity']
        except:
            NSuns = exp_info['Excitation Intensity']
        indexes.append('NSuns')
        values.append(NSuns)

    # Get sample composition for A site and X site
    with open(folder_path + '\\sample_info.json') as samp_json:
        samp_info = json.load(samp_json)
        site_a = parse_sample_comp(samp_info['Starting Composition A-site'], 'A', comp_regex)  # [species_list, values_list]
        site_x = parse_sample_comp(samp_info['Starting Composition X-site'], 'X', comp_regex)
        indexes += site_a[0] + site_x[0]
        values += site_a[1] + site_x[1]
    
    # DEBUGS IN DF
#     indexes.append('File Loc.')
#     values.append(f'{folder_path}')
    
    
    return pd.Series(values, indexes)
    

In [183]:
# Load all feature and target data into dataframe df
data = []
study_folders = os.listdir(source_path)
for study in study_folders:
    run_folders = os.listdir(source_path + '\\' + study)
    for run in run_folders:
        try:
            data.append(get_single_run_data(f'{source_path}\\{study}\\{run}', 300))
        except Exception as e:
            print(e)
            
df = pd.DataFrame(data)

C:\Users\andre\Documents\replicate_data\190916_PLVA_Perovskite_Degradation_Study\PL_DF_T_MAPI_air_60RH_25C_8Suns reaches LD75 before the end of the prediction window.
C:\Users\andre\Documents\replicate_data\190916_PLVA_Perovskite_Degradation_Study\PL_PC_T_MAPI_wetN2_60RH_25C_8Suns is not in 21% O2.
'Low Freq LD [norm]'
C:\Users\andre\Documents\replicate_data\191203_Single_Sample_PL_PC_T_Combinatorial_Stress_Study\PL_PC_T_MAPI_8Sun_25C_60RH_100pctO2_Unencap is not in 21% O2.
'Low Freq LD [norm]'
'Low Freq LD [norm]'
C:\Users\andre\Documents\replicate_data\200525_Replication_Study\PL_PC_T_MAPI_8Sun_25C_60RH_air_28_50x does not reach LD75.
'Low Freq LD [norm]'
'Low Freq LD [norm]'
C:\Users\andre\Documents\replicate_data\200709_Modeling_Runs\PC_PL_Tr_MAPI_8sun_25C_60RH_N2_2 is not in 21% O2.
C:\Users\andre\Documents\replicate_data\200709_Modeling_Runs\PC_PL_Tr_MAPI_8sun_25C_60RH_N2_3 is not in 21% O2.


In [184]:
pd.set_option('display.max_columns', None)

# TEMPORARILY DROPPING ROWS 0 AND 17 BECAUSE THEY ARE THE <2 AND ~7 OUTLIERS, RESPECTIVELY not necessarily the correct rows anymore
# df.drop([0,17], axis=0, inplace=True) # Removed this for now because the conditions in filling the dataframe are throwing it off
df

,ln(t_LD75),xy0t0 t0,PLQY_xy0t0 t0,xy1t0 t0,xy2t0 t0,xy3t0 t0,QFLS_xy0t0 t0,frac_bright t0,cv_slopes t0,dTr0,dPL0,dLd0,ddTr0,ddPL0,ddLd0,"Tr [norm], t=120 s","PL [norm], t=120 s","Ld [norm], t=120 s","Tr [norm], t=240 s","PL [norm], t=240 s","Ld [norm], t=240 s","Tr [norm], t=300 s","PL [norm], t=300 s","Ld [norm], t=300 s",RH (%),Temp (C),Atm O2 (%),NSuns,MA,I
0,4.731132,2.641097e+18,0.000211,4.766278e+17,-1.526802,1.423006,1.168810,0.009617,0.484497,0.000248,-0.000960,7.234403e-05,8.323522e-07,0.000009,0.000002,1.017722,0.755709,0.975882,1.059416,0.769702,1.017363,1.074630,0.792354,1.005480,59.5,25.0,21.0,8.0,1.0,1.0
1,4.669558,1.761538e+19,0.001408,4.531460e+18,-0.776540,-0.587217,1.217866,0.512816,0.642370,0.000068,0.005583,-3.645883e-05,NaN,NaN,NaN,1.008214,1.669937,0.995625,1.016428,2.339874,0.991250,1.020535,2.674843,0.989062,60.0,25.0,21.0,8.0,1.0,1.0
2,4.136253,5.290780e+19,0.004230,1.066053e+19,0.431559,-1.517979,1.246297,0.997114,0.844637,0.000258,0.006022,7.586888e-05,NaN,NaN,NaN,1.030957,1.722623,1.009104,1.061915,2.445246,1.018209,1.077394,2.806557,1.022761,60.0,25.0,21.0,8.0,1.0,1.0
3,2.861123,1.212450e+18,0.000097,1.402880e+17,-0.156370,-1.873200,1.148683,0.718168,0.399521,0.000584,0.008668,4.106729e-04,NaN,NaN,NaN,1.070020,2.040166,1.049281,1.140040,3.080332,1.098562,1.175051,3.600414,1.123202,60.0,25.0,21.0,8.0,1.0,1.0
4,4.837339,2.362931e+18,0.000189,5.366656e+17,-1.006490,0.347070,1.165933,0.871570,0.596208,0.000064,0.010952,8.426585e-05,NaN,NaN,NaN,1.007636,2.314183,1.010112,1.015272,3.628366,1.020224,1.019090,4.285458,1.025280,60.0,25.0,21.0,8.0,1.0,1.0
5,5.200043,2.832169e+19,0.002264,6.670230e+18,-0.869977,0.378392,1.230142,0.853795,0.639707,0.000017,0.002833,-2.335286e-05,NaN,NaN,NaN,1.002032,1.339901,0.997198,1.004064,1.679801,0.994395,1.005080,1.849751,0.992994,60.0,25.0,21.0,8.0,1.0,1.0
6,5.220718,4.556241e+19,0.003643,1.002666e+19,-0.240510,-1.473940,1.242433,0.904402,0.745533,0.000044,0.008225,1.351494e-04,NaN,NaN,NaN,1.005282,1.986998,1.016218,1.010563,2.973997,1.032436,1.013204,3.467496,1.040545,60.0,25.0,21.0,8.0,1.0,1.0
7,5.189162,2.615572e+19,0.002091,9.319970e+18,-0.455374,-0.484532,1.228085,0.882315,0.643260,0.000025,0.006390,-1.392503e-04,NaN,NaN,NaN,1.002982,1.766805,0.983290,1.005964,2.533610,0.966580,1.007456,2.917012,0.958225,60.0,25.0,21.0,8.0,1.0,1.0
8,5.128772,1.735346e+19,0.001387,2.791081e+18,1.649056,0.926015,1.217478,0.770062,0.750827,0.000038,0.005749,1.508622e-04,NaN,NaN,NaN,1.004613,1.689852,1.018103,1.009226,2.379705,1.036207,1.011533,2.724631,1.045259,60.0,25.0,21.0,8.0,1.0,1.0
9,4.989090,4.105706e+18,0.000328,1.669929e+18,0.926307,-0.761915,1.180215,0.327611,0.496917,0.000011,0.001717,2.311654e-04,NaN,NaN,NaN,1.001274,1.206046,1.027740,1.002549,1.412092,1.055480,1.003186,1.515115,1.069350,60.0,25.0,21.0,8.0,1.0,1.0


In [170]:
df['File Loc.'][10]

'C:\\Users\\andre\\Documents\\replicate_data\\200525_Replication_Study\\PL_PC_T_MAPI_8Sun_25C_60RH_air_1_1'

In [135]:
df.reset_index(drop=True, inplace=True)
df

""


# Characterizing Variance <a class="anchor" id="char_var"></a>

First some numerical measures of the variance in t$_{\text{LD75}}$, then some visualizations.

In [ ]:
def Characterize_Variable_Variance(group_df, variable):
    median = group_df[variable].median()
    mean = group_df[variable].mean()
    variance = group_df[variable].var()
    std_dev = group_df[variable].std()
    
    print(f'Statistical Measurements of {variable} for Group 1:')
    print(f'\tMedian: {median:.3f}')
    print(f'\tMean: {mean:.3f}')
    print(f'\tStandard Deviation: {std_dev:.3f}')
    print(f'\tVariance: {variance:.3f}')

In [ ]:
Characterize_Variable_Variance(df, 'ln(t_LD75)')

## Histogram Shows Distribution of data

In [ ]:
fig, ax = plt.subplots(1,1,figsize=(10,5))
ax.set_title('Distribution of ln(t_LD75) Data', fontsize=15)
ax.set_xlabel('ln(t_LD75) [min]', fontsize=15)
ax.set_ylabel('Frequency', fontsize=15)
df['ln(t_LD75)'].hist(bins=20, ax=ax)

## Feature Box Plot
Next up is a box plot of each column to show generally how much each possible feature varies within the group.

KIND OF WORTHLESS RIGHT NOW

In [ ]:
fig, ax = plt.subplots(1,1,figsize=(15, 5))
df.plot(kind='box', ax=ax)

## Interactive Scatter Plot, $t_{\text{LD75}}$ vs. Features

In [ ]:
from ipywidgets import interact

def my_scatter(x=df.drop('ln(t_LD75)', axis=1).columns):
    fig, ax = plt.subplots(1,1,figsize=(8,8))
    ax.set_ylabel('ln(t_LD75) [min]', fontsize=15)
    ax.set_xlabel(x, fontsize=15)
    ax.scatter(df[x], df['ln(t_LD75)'])

In [ ]:
interact(my_scatter)

# Feature Explanation <a class="anchor" id="feature"></a>

* Incident_Flux t0
* xy0t0 t0
* PLQY_xy0t0 t0
* xy1t0 t0
* xy2t0 t0
* xy3t0 t0
* QFLS_xy0t0 t0
* Low Freq LD [nm] t0
* frac_bright t0
* cv_slopes t0
* dTr0
* dPL0
* dLd0
* ddTr0
* ddPL0
* ddLd0
* Tr [norm], t1 (t2, ...)
* PL [norm], t1 (t2, ...)
* Ld [norm], t1 (t2, ...)


# Modeling with LASSO Regression <a class="anchor" id="LASSO"></a>

In [9]:
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import Lasso
from sklearn.model_selection import GridSearchCV
from sklearn.utils._testing import ignore_warnings
from sklearn.exceptions import ConvergenceWarning

In [10]:
def choose_feature_set(df, feature_set_name, time_allowed):
    # Take one of several strings for possible models
    # '+video', '+timeseries', '+both'
    X = df.drop(['ln(t_LD75)', 'RH (%)', 'Temp (C)', 'Atm O2 (%)', 'NSuns', 'MA', 'I'], axis=1)
    
    timeseries = [f'Tr [norm], t={time_allowed} s', f'PL [norm], t={time_allowed} s', f'Ld [norm], t={time_allowed} s']
    video = ['xy0t0 t0', 'xy1t0 t0', 'xy2t0 t0', 'xy3t0 t0', 'frac_bright t0', 'cv_slopes t0']
#     for i in range(1,6):
#         timeseries.append(f'Tr [norm], t{i}')
#         timeseries.append(f'PL [norm], t{i}')
#         timeseries.append(f'Ld [norm], t{i}')
    
    if feature_set_name == '+video':
        X = X.drop(timeseries, axis=1)
    
    if feature_set_name == '+timeseries':
        X = X.drop(video, axis=1)
        
    return X

In [11]:
# Break df into X (features) and y (target), standard scale features
X = choose_feature_set('+video', 60)
y = df['ln(t_LD75)']

scaler = StandardScaler()
X = pd.DataFrame(scaler.fit_transform(X), columns=X.columns)

TypeError: choose_feature_set() missing 1 required positional argument: 'time_allowed'

In [ ]:
X

For the modeling, we use our Leave One Out validation scheme since our data is small and will evaluate error using the model of median error.

In [151]:
def feature_weight_plot(feature_weights, feature_labels):
    # Create bar chart of coefficient weights
    feature_weights, feature_labels = zip(*sorted(zip(feature_weights, feature_labels))) 
    fig, ax = plt.subplots(tight_layout=True)
    ax.bar(feature_labels, feature_weights, width=0.4)
    ax.set_ylabel('Feature Weight')
    plt.xticks(rotation=90)
    plt.show()

In [152]:
def parity_plot(model, X_test, X_train, y_test, y_train):
    # Plot y_pred vs y_test (which is the actual value), add line y = x for reference
    y_test_pred = model.predict(X_test)
    y_train_pred = model.predict(X_train)
    
    plt.scatter(y_train, y_train_pred, label='Train') # Plotting predictions of training points in a one color
    plt.scatter(y_test, y_test_pred, label='Test')    # Plotting prediction of test point in another color
    plt.ylabel('Predicted ln(t_LD75)')
    plt.xlabel('Actual ln(t_LD75)')
    plt.title('Median Abs Error = ' + str(abs(model.predict(X_test)[0] - y_test)))
    
    _min = min(y_train.min(), y_train_pred.min(), y_test, y_test_pred)
    _max = max(y_train.max(), y_train_pred.max(), y_test, y_test_pred)
    plt.xlim(_min*0.9, _max + _min*0.1)
    plt.ylim(_min*0.9, _max + _min*0.1)
    plt.plot([_min*0.9, _max + _min*0.1], [_min*0.9, _max + _min*0.1]) # Plots parity line

    plt.legend()
    plt.show()

In [153]:
def model_report(model_i, feature_labels):
    feature_weight_plot(model_i.coef_, feature_labels)
    parameters = model_i.get_params()
    print('\n-----Parameters for Model-----\n')
    for k in parameters.keys():
        print(f'{k}'.ljust(28) + f'{parameters[k]}')
    print('\n-----Feature Weights-----\n')
    print(pd.Series(model_i.coef_, index=feature_labels))

In [154]:
# @ignore_warnings(category=ConvergenceWarning)
def model_single_point(X, y, index, plot_model=False):
    # Set up training and testing data for this point
    X_train_i = X.drop(index, axis=0)   # Dataframe (column labels are features, row labels are indices)
    y_train_i = y.drop(index, axis=0)   # Series (labels are indices 0-..., values are y values)
    X_test_i = X.loc[index].values.reshape(1, -1)     # Series (labels are feature names, values are feature values) to array with .values, reshaped because single sample
    y_test_i = y.loc[index]     # float64

    # Train model
    param_grid = {'alpha': np.logspace(-8, 1, num=100), 'fit_intercept': [True], 'normalize': [False]}
    grid = GridSearchCV(Lasso(max_iter=1e7), param_grid, cv=5)
    grid.fit(X_train_i, y_train_i)
    model_i = grid.best_estimator_
    
    # Get the absolute value error for this model
    abs_error_i = abs(model_i.predict(X_test_i)[0] - y_test_i)

    if plot_model: # Set to true to plot model parity, mainly used for median error model
        parity_plot(model_i, X_test_i, X_train_i, y_test_i, y_train_i)
    return abs_error_i, model_i

In [155]:
def LOO_procedure(X, y):
    # Find abs error for model based on every point
    models = []
    abs_errors = []
    all_feature_weights = []
    for i in range(len(X)):
        error_i, model_i = model_single_point(X, y, i, False) # Set to True if you want to plot every model just to see
        models.append(model_i)
        abs_errors.append(error_i)
        feature_weights_i = model_i.coef_
        all_feature_weights.append(feature_weights_i)
        if i % 3 == 0:
            print(f'Progress: {i} / {len(X)-1}')
    
    # Get list of mean feature weights
    mean_feature_weights = []
    for i in range(len(all_feature_weights[0])):
        feat = 0
        for j in range(len(all_feature_weights)):
            feat = feat + all_feature_weights[j][i]
        feat = feat / len(all_feature_weights)
        mean_feature_weights.append(feat)

    # Find index that has the median abs error
    median_index = np.argsort(abs_errors)[len(abs_errors) // 2]
    median_error = abs_errors[median_index]
    median_model = models[median_index]
    
    # Display parity plot for median error model, give parameters
    model_single_point(X, y, median_index, True)
    parameters = median_model.get_params()
    print('\n-----Parameters for Model-----\n')
    for k in parameters.keys():
        print(f'{k}'.ljust(28) + f'{parameters[k]}')
    print('\n-----Median Model Feature Weights-----\n')
    print(pd.Series(median_model.coef_, index=X.columns))
    feature_weight_plot(median_model.coef_, X.columns)
    
    # Give and plot values of mean feature weights
    print('\n----Mean Feature Weights----\n')
    print(pd.Series(mean_feature_weights, index=X.columns))
    feature_weight_plot(mean_feature_weights, X.columns)
    
    
    # Heat map of all feature weights
    plt.figure(figsize=(10,10))
    sns.heatmap(pd.DataFrame(all_feature_weights, columns=X.columns))
    
    return median_index, mean_feature_weights, median_model, median_error

## Summary of LASSO Results

Below is the parity plot for the model of median error, the parameters and feature weights of that median model, and a plot of the mean feature weights.

In [186]:
def feature_set_time_model_combo(feature_set_str, time_allowed):
    print(f'{feature_set_str} FEATURE SET, {time_allowed} s ALLOWED FOR DATA COLLECTION')
    print('-----------------------------------------------------------------------------')
    print('-----------------------------------------------------------------------------')
    # feature_set_str can be '+video', '+timeseries', '+both'
    
    # FILL DATAFRAME WITH EVERYTHING BASED ON TIME ALLOWED (in seconds, makes things easier)
    data = []
    study_folders = os.listdir(source_path)
    for study in study_folders:
        run_folders = os.listdir(source_path + '\\' + study)
        for run in run_folders:
            try:
                data.append(get_single_run_data(f'{source_path}\\{study}\\{run}', time_allowed))
            except:
#                 print(f'Error in: {study}/{run}')
                pass
    
    df = pd.DataFrame(data)
    df.drop([10], axis=0, inplace=True)
    df.reset_index(drop=True, inplace=True)
    
    df.dropna(axis='columns', inplace=True)
    
    # CHOOSE FEATURES BASED ON FEATURE SET
    
    # Break df into X (features) and y (target), standard scale features
#     X = choose_feature_set(df, feature_set_str, time_allowed)
    X = df.drop(['ln(t_LD75)', 'RH (%)', 'Temp (C)', 'Atm O2 (%)', 'NSuns', 'MA', 'I'], axis=1)
    y = df['ln(t_LD75)']
    
    print(X)
    scaler = StandardScaler()
    X = pd.DataFrame(scaler.fit_transform(X), columns=X.columns)
    
    
    # RUN LOO PROCEDURE TO GET MEDIAN MODEL/ERROR/FEATURE WEIGHTS/PARAMETERS, ALL FEATURE WEIGHTS
    median_index, mean_feature_weights, median_model, median_error = LOO_procedure(X, y)

    print('\n\n\n')
    # RETURN MEDIAN ERROR SO WE CAN PLOT AGAINST TIME_ALLOWED
    return median_error

In [ ]:
median_errors = []
times_allowed = np.arange(0, 600, 60)

for t in times_allowed:
    median_errors.append(feature_set_time_model_combo('+both', t))

In [187]:
temp = feature_set_time_model_combo('+both', 300)

+both FEATURE SET, 300 s ALLOWED FOR DATA COLLECTION
-----------------------------------------------------------------------------
-----------------------------------------------------------------------------
        xy0t0 t0  PLQY_xy0t0 t0      xy1t0 t0  xy2t0 t0  xy3t0 t0  \
0   2.641097e+18       0.000211  4.766278e+17 -1.526802  1.423006   
1   1.761538e+19       0.001408  4.531460e+18 -0.776540 -0.587217   
2   5.290780e+19       0.004230  1.066053e+19  0.431559 -1.517979   
3   1.212450e+18       0.000097  1.402880e+17 -0.156370 -1.873200   
4   2.362931e+18       0.000189  5.366656e+17 -1.006490  0.347070   
5   2.832169e+19       0.002264  6.670230e+18 -0.869977  0.378392   
6   4.556241e+19       0.003643  1.002666e+19 -0.240510 -1.473940   
7   2.615572e+19       0.002091  9.319970e+18 -0.455374 -0.484532   
8   1.735346e+19       0.001387  2.791081e+18  1.649056  0.926015   
9   4.105706e+18       0.000328  1.669929e+18  0.926307 -0.761915   
10  6.401794e+18       0.000512 

Progress: 0 / 26
Progress: 3 / 26


c:\users\andre\appdata\local\programs\python\python38-32\lib\site-packages\sklearn\linear_model\_coordinate_descent.py:530: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 0.12546175468288778, tolerance: 0.002795265705552559
  model = cd_fast.enet_coordinate_descent(
c:\users\andre\appdata\local\programs\python\python38-32\lib\site-packages\sklearn\linear_model\_coordinate_descent.py:530: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 0.12546181635420117, tolerance: 0.002795265705552559
  model = cd_fast.enet_coordinate_descent(
c:\users\andre\appdata\local\programs\python\python38-32\lib\site-packages\sklearn\linear_model\_coordinate_descent.py:530: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 0.12546188911247347, tolerance: 0.002795265705552559
  model = cd_fast.enet_coordinate_descent

KeyboardInterrupt: 